# Mastering the Art of Multitasking
## A Deep Dive into Concurrent Programming with Python
**Date**: December 3rd, 2024<br/>
**Instructor**: Jan Janak &lt;janakj@cs.columbia.edu&gt;

**Disclaimer**: This lecture is work in progress. The content may change significantly in future iterations.

<hr/>

In [2]:
%%html
<style>
    .fig {
        display: flex;
        justify-content: space-around;
    }
</style>

In [22]:
%load_ext autoreload

In [24]:
print("hello2")

hello


In [3]:
import ipywidgets as widgets

# About this Notebook

This Jupyter notebook is similar to those that you may have seen in other courses like ENGI E1006 and COMS W2132. It is different from those notebooks in the following ways:

  1) The notebook (its Python kernel) runs on a Raspberry Pi computer embedded in the robot
  1) It has access to the robot's hardware peripheral's (3D mouse, speaker, microphone, servos, LEDs)
  1) It has the various Python libraries for accessing the hardware preinstalled
  1) It has access to the API keys (under my account) for Google Cloud and OpenAI's ChatGPT API

When I edit the code cells in this notebook during the lecture, I edit a running Python program on the robot. Unfortunately, you will not be able to re-run the notebook on your laptop because it does not have the required hardware.

All the software used in this notebook is open-source. Check out the [GitHub repository](https://github.com/janakj/shaky-steve) or the robot's [home page](https://janakj.github.io/shaky-steve/).

# The Setup

The following diagram illustrates the setup:

<div class="fig">
    <img width=1024 src="../images/setup.png"/>
</div>

  * The Jupyter notebook runs directly on the robot
  * I access the notebook from the browser on my presentation laptop via Wi-FI
  * The robot communicates with cloud services over the Internet:
     * Google Cloud for the Speech-to-Text and Text-to-Speech services
     * OpenAI Cloud for the ChatGPT API

# Additional Resources

  * Shaky Steve's Python software is open-source. You can study (and improve) it at https://github.com/janakj/shaky-steve.
  * Steve also has a home page at https://janakj.github.io/shaky-steve/.

Shaky Steve provides a simple API consisting of just a few functions. All the functions can be imported from the module "steve".
  * set_led(red, green, blue)
  * 

In [4]:
torso    = widgets.FloatSlider(min=-90  , max=90  , step=0.1, description='Torso'  ,  readout=True, readout_format='.1f', continuous_update=False)
clamp    = widgets.FloatSlider(min=0    , max=1   , step=0.1, description='Clamp'  ,  readout=True, readout_format='.1f', continuous_update=False)
wrist_lr = widgets.FloatSlider(min=-90  , max=90  , step=0.1, description='Wrist ⇆',  readout=True, readout_format='.1f', continuous_update=False)
wrist_ud = widgets.FloatSlider(min=-90  , max=90  , step=0.1, description='Wrist ⇅',  readout=True, readout_format='.1f', continuous_update=False)
elbow    = widgets.FloatSlider(min=-58.4, max=61.7, step=0.1, description='Elbow',    readout=True, readout_format='.1f', continuous_update=False)
shoulder = widgets.FloatSlider(min=-90  , max=90  , step=0.1, description='Shoulder', readout=True, readout_format='.1f', continuous_update=False)

display(torso)
display(clamp)
display(wrist_lr)
display(wrist_ud)
display(elbow)
display(shoulder)

FloatSlider(value=0.0, continuous_update=False, description='Torso', max=90.0, min=-90.0, readout_format='.1f'…

FloatSlider(value=0.0, continuous_update=False, description='Clamp', max=1.0, readout_format='.1f')

FloatSlider(value=0.0, continuous_update=False, description='Wrist ⇆', max=90.0, min=-90.0, readout_format='.1…

FloatSlider(value=0.0, continuous_update=False, description='Wrist ⇅', max=90.0, min=-90.0, readout_format='.1…

FloatSlider(value=0.0, continuous_update=False, description='Elbow', max=61.7, min=-58.4, readout_format='.1f'…

FloatSlider(value=0.0, continuous_update=False, description='Shoulder', max=90.0, min=-90.0, readout_format='.…

In [5]:
from time import sleep
import asyncio

async def anim1():
    for i in range(0, 90, 1):
        shoulder.value = float(i)
        await asyncio.sleep(0.1)

async def anim2():
    for i in range(0, 90, 1):
        torso.value = float(i)
        await asyncio.sleep(0.2)

asyncio.create_task(anim1())
asyncio.create_task(anim2())

<Task pending name='Task-6' coro=<anim2() running at /var/folders/6p/gzq7y7116d982z30cf944kpm0000gn/T/ipykernel_78303/2152007556.py:9>>

In [6]:

async def high_five():
    torso.value = -0.429
    shoulder.value = 18.799
    elbow.value = 61.7
    wrist_ud.value = -5.632
    wrist_lr.value = 0.282
    clamp.value = 0.03

    await asyncio.sleep(2)
    for i in range(5):
        clamp.value = 0
        await asyncio.sleep(0.2)
        clamp.value = 1
        await asyncio.sleep(0.2)

await high_five()

In [7]:
leds = []
for i in range(4):
    leds.append(widgets.HTML(value=
'''
<div style="display: flex; justify-content: center; align-items: center; width: 70px; height: 70px; background-color: black; border-radius: 100%">
    <div style="margin-top: -2px; font-size: 70px; text-align: center; line-height: 1">&#9679;</div>
</div>
'''))

widgets.HBox(leds)

In [9]:
l = [1, 2, 3]

In [ ]:
leds[0].

In [209]:
for led in leds:
    led.style.text_color = '#ffffff'

In [212]:
import math

async def breathe(period=4, gamma=0.12, rate=10):
    n = rate * period
    while True:
        for i in range(n):
            v = math.exp(-(pow((i / n - 0.5) / gamma, 2)) / 2)
            R = r * v
            G = g * v
            B = b * v
            leds[0].style.text_color = f'#{R:02x}{G:02x}{B:02x}'
            await asyncio.sleep(1 / rate)

In [213]:
await breathe()

ValueError: Unknown format code 'x' for object of type 'float'

In [ ]:
import asyncio
asyncio.all_tasks()

In [ ]:
task, = [task for task in asyncio.all_tasks() if task.get_name() == 'Task-5']
task.cancel()